In [125]:
import os
import pandas as pd
import numpy as np
from natsort import natsorted
from sklearn import metrics

In [126]:
data_path = "/Users/imrankabir/Desktop/research/vqa_accessibility/Dashboard-For-VQA/Dashboard Data"
gt_fol = os.path.join(data_path, "GT")

In [227]:
def get_mAP(targets, preds):
    APs = np.zeros(preds.shape[1])
    for k in range(preds.shape[1]):
        APs[k] = _average_precision_2(targets[:, k], preds[:, k])

    return APs


def _average_precision_2(target, output):
    epsilon = 1e-8

    total_pred_pos = np.count_nonzero(output)

    TP = np.count_nonzero(np.logical_and(output, target))
    FP = total_pred_pos - TP

    AP = TP / (TP + FP + epsilon)

    return AP


def calculate_model_map(gt_path, pred_path):
    gt_dict = {}
    pred_dict = {}
    
    gt_files = [x for x in os.listdir(gt_path) if x.endswith('.csv') and int(x.split('-')[1])<=16]

    for fl in gt_files:
        if fl in ["video-10-segment-1.csv", "video-9-segment-2.csv"]:
            continue
        gt_fl = os.path.join(gt_path, fl)
        pred_fl = os.path.join(pred_path, fl)
        if not os.path.exists(pred_fl):
            continue

        gt_data = pd.read_csv(gt_fl)
        gt_data.replace(-1, 1, inplace=True) 
        x = []
        for index, row in gt_data.iterrows():
            if list(row)[0].strip() in ["Sidewalk pits"]:
                continue
            if list(row)[0].strip() in gt_dict.keys():
                gt_dict[list(row)[0].strip()] += list(row)[1:]
            else:
                gt_dict[list(row)[0].strip()] = list(row)[1:]

            x.append(len(list(row)[1:]))

        y = []    
        pred_data = pd.read_csv(pred_fl)
        pred_data.replace(-1, 1, inplace=True) 
        for index, row in pred_data.iterrows():
            if list(row)[0].strip() in ["Sidewalk pits"]:
                continue
            if list(row)[0].strip() in pred_dict.keys():
                pred_dict[list(row)[0].strip()] += list(row)[1:]
            else:
                pred_dict[list(row)[0].strip()] = list(row)[1:]
            y.append(len(list(row)[1:]))
            
        if x != y:
            print(len(x), len(y), fl)
    
    target_array = []
    pred_array = []

    for key in gt_dict.keys():
        if key in pred_dict.keys():
            if len(gt_dict[key]) == len(pred_dict[key]):
                target_array.append(gt_dict[key])
                pred_array.append(pred_dict[key])
            else:
                print(len(gt_dict[key]), len(pred_dict[key]), key)

    target_array = np.array(target_array).T
    pred_array = np.array(pred_array).T
    
    try:
        # ap = np.array(metrics.precision_score(target_array, pred_array, average=None))
        ap_2 = get_mAP(target_array, pred_array)
    except:
        # ap = 0
        ap_2 = 0
        print(target_array.shape, pred_array.shape)
    
    return ap_2, ap_2

In [228]:
pred_fol = os.path.join(data_path, "BLIP")
apg, apg_2 = calculate_model_map(gt_fol, pred_fol)

print(apg.mean(), apg_2.mean())

0.23132030614136453 0.23132030614136453


In [229]:
pred_fol = os.path.join(data_path, "GPV-1")
apg, apg_2 = calculate_model_map(gt_fol, pred_fol)

print(apg.mean(), apg_2.mean())

0.19984905268298686 0.19984905268298686


In [230]:
pred_fol = os.path.join(data_path, "yolo_v7")
apg, apg_2 = calculate_model_map(gt_fol, pred_fol)

print(apg.mean(), apg_2.mean())

0.1019004699310333 0.1019004699310333


In [231]:
pred_fol = os.path.join(data_path, "HRNet_V2")
apg, apg_2 = calculate_model_map(gt_fol, pred_fol)

print(apg.mean(), apg_2.mean())

0.08170082537543284 0.08170082537543284


In [232]:
pred_fol = os.path.join(data_path, "mask_rcnn")
apg, apg_2 = calculate_model_map(gt_fol, pred_fol)

print(apg.mean(), apg_2.mean())

0.056110478031014654 0.056110478031014654


In [233]:
pred_fol = os.path.join(data_path, "faster_rcnn")
apg, apg_2 = calculate_model_map(gt_fol, pred_fol)

print(apg.mean(), apg_2.mean())

0.05574902585860564 0.05574902585860564


In [234]:
pred_fol = os.path.join(data_path, "Random")
apg, apg_2 = calculate_model_map(gt_fol, pred_fol)

print(apg.mean(), apg_2.mean())

0.13666928138079906 0.13666928138079906
